In [7]:
import torch
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
from train import SimCLR_Train
from models import SimCLR_Model

In [8]:
# credit: https://github.com/kuangliu/pytorch-cifar/issues/19
# for CIFAR-10 normalization values
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

batch_size_list = [256, 512, 1024, 2048, 4096]

# the maximum batchsize is 1024
batch_size = batch_size_list[2]

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
resnet20 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = SimCLR_Model(base_encoder=resnet20)
net = net.to(device)

Using cache found in C:\Users\Koir/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


In [4]:
# use adam/sgd since we don't have large batches
# possible config for hyperparams:
# temp = [0.1, 0.5, 1.0]
# lr = 1e-4
# epoch = [100, 200, 300]
# could add a cosine scheduler
# check here for more config: https://github.com/p3i0t/SimCLR-CIFAR10
train_loss_list = SimCLR_Train(net, device, batch_size, train_loader, num_epoch=100, temp=0.5, lr=1e-4, optim='adam', lr_scheduler='None')

Epoch 1


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [01:56<00:00,  2.38s/it]


Epoch: 1, Training Loss: 7406799.0
Epoch 2


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [02:11<00:00,  2.69s/it]


Epoch: 2, Training Loss: 7123202.0
Epoch 3


...

Epoch: 98, Training Loss: 6702457.5
Epoch 99


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [02:02<00:00,  2.51s/it]


Epoch: 99, Training Loss: 6702174.0
Epoch 100


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [01:56<00:00,  2.38s/it]

Epoch: 100, Training Loss: 6697732.0
Finish training, saving model...


In [ ]:
# linear evaluation code--rerun this block for linear eval

from train import Linear_Eval_Train
from models import Linear_Eval
resnet20 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = SimCLR_Model(base_encoder=resnet20)
net = net.to(device)
net.load_state_dict(torch.load('SimCLR_batchsize=1024_lr=0.0001_optim=adam_temp=0.5_epoch=100.pt'))

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

batch_size = 512

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

base = net.base_encoder
linear_eval = Linear_Eval(base)
linear_eval = linear_eval.to(device)
loss_list, acc_list = Linear_Eval_Train(linear_eval, device, train_loader, num_epoch=100, lr=0.1, optim='sgd', lr_scheduler='cosine')

In [ ]:
# test linear eval
linear_eval.eval()
test_acc = 0
for batch in test_loader:
    image, label = batch
    image, label = image.to(device), label.to(device)
    loss_func = nn.CrossEntropyLoss()
    actual_batch_size = image.shape[0]
    pred = linear_eval(image)
    loss = loss_func(pred, label)
    test_acc += torch.sum(torch.argmax(pred, dim=-1) == label) / actual_batch_size
test_acc /= len(test_loader)
print('Test Acc: ', test_acc.item())